# 笔记——深度学习预备知识
## 前言
**文中部分内容来源于《动手学深度学习》，官方电子书网址：[《动手学深度学习》 — 动手学深度学习 2.0.0-beta0 documentation (d2l.ai)](http://zh-v2.d2l.ai/index.html)**

## 一、数据操作

### 广播机制

广播机制解决的是不同形状的张量之间的元素操作。通常来说，形状不同的矩阵亦或者说张量，是不能做加法的。广播机制就可以做到。

In [11]:
import torch
a = torch.arange(12).reshape(3,-1)
print(a)
a += 3
print(a)

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])
tensor([[ 3,  4,  5,  6],
        [ 7,  8,  9, 10],
        [11, 12, 13, 14]])


### 原地操作

其实我个人认为节省内存这一问题不在于节省内存。在于当神经网络优化的时候引用到了旧的参数，这会带来一些问题。于是我们更希望的不是对操作后的参数重新开辟一个内存空间，然后再对这个空间里的参数优化（所谓优化网络时引用到了旧的参数地址，可能就是体现在我们没有引用到新开辟的内存里的参数），而是希望就地优化、就地执行对参数的优化。

《动手学深度学习》中的作者提供了两种方式：

**（1）提前分配空间**

In [12]:
a = torch.arange(12).reshape(3,-1)
b = torch.randn(3,4) 
z = torch.zeros_like(b)
print(id(z))
z[:] = a+b
print(id(z))

140452025062592
140452025062592


**最重要的是：z[:]而不是z=a+b，不然地址是不一样的**

In [13]:
a = torch.arange(12).reshape(3,-1)
b = torch.randn(3,4)
z = torch.zeros_like(b)
print(id(z))
z = a+b
print(id(z))

140452025276928
140457779932480


**（2）用“+=”运算符**

如果忘了或者不习惯z[:]=a+b，就使用“+=”运算符。

In [14]:
a = torch.randn(3,4)
b = torch.randn(3,4)
print(id(a), a)
a += b
print(id(a), a)

140452025276224 tensor([[ 1.3874, -0.9760,  1.4103, -0.9757],
        [-0.5423,  0.0741,  0.3826,  0.8261],
        [ 0.1372, -0.2280, -1.3714,  0.0853]])
140452025276224 tensor([[-0.1606, -1.7621,  1.1956,  1.4307],
        [-2.8320, -0.1447,  0.6095,  0.4477],
        [-0.5403,  1.6106, -1.6889, -0.1384]])


### 转换为Python对象
Numpy->Tensor
Tensor->Numpy

In [15]:
x = torch.randn(4,4)
np = x.numpy()
tensor = torch.tensor(np)
type(np),type(tensor)

(numpy.ndarray, torch.Tensor)

## 二、数据预处理
创建人工数据集。

In [16]:
import os
os.makedirs(os.path.join('..', 'data'), exist_ok=True)
data_file = os.path.join('..', 'data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('NumRooms,Alley,Price\n')  # 列名
    f.write('NA,Pave,127500\n')  # 每行表示一个数据样本
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

In [17]:
import pandas as pd
data = pd.read_csv(data_file) 
data

,NumRooms,Alley,Price
0,NaN,Pave,127500
1,2.0,NaN,106000
2,4.0,NaN,178100
3,NaN,NaN,140000


### 处理缺失值
数据科学很重要的一点就是处理缺失的数据。典型的办法是插值或删除。下面考虑的插值。

In [18]:
inputs, outputs = data.iloc[:, 0:2], data.iloc[:, 2]
inputs = inputs.fillna(inputs.mean())
print(inputs)

   NumRooms Alley
0       3.0  Pave
1       2.0   NaN
2       4.0   NaN
3       3.0   NaN


<ipython-input-18-7f3bcd4dc11f>:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  inputs = inputs.fillna(inputs.mean())


对于不是数值类型的缺失数据，可以把缺失的指做成一个特别的类。缺失值也是一种信息。如下，将Alley中的两种不同信息做成两个类。

In [19]:
inputs = pd.get_dummies(inputs, dummy_na=True)
inputs

,NumRooms,Alley_Pave,Alley_nan
0,3.0,1,0
1,2.0,0,1
2,4.0,0,1
3,3.0,0,1


当我们把缺失的数据补全/非数值的信息通过数值表征以后，我们就可以将这些信息转化为一个tensor

In [20]:
x, y = torch.tensor(inputs.values), torch.tensor(outputs.values)
x, y

(tensor([[3., 1., 0.],
         [2., 0., 1.],
         [4., 0., 1.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([127500, 106000, 178100, 140000]))

## 三、有意思QA
### view和reshape和函数的区别。
### array和tensor的区别。